In [ ]:
#WORKING PROJECT
import numpy as np
import argparse
import pickle
import cv2
import os
import time
from keras.models import load_model
from collections import deque
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import folium
from geopy.geocoders import Nominatim



def find_loc():
    # Create a geocoder object
    geolocator = Nominatim(user_agent="my-app")

    # Specify the location you want to geocode
    location = "India"

    # Geocode the location
    geocode_result = geolocator.geocode(location)

    # Print the place name, latitude, and longitude
    lat=geocode_result.latitude
    long=geocode_result.longitude
    loc(lat,long)

def loc(lat,long):
    map = folium.Map(location=[lat, long], zoom_start=15)
    folium.Marker(location=[lat, long], popup="Location").add_to(map)
    map.save("map.html")
#from google.colab.patches import cv2_imshow
#fig=plt.figure(figsize=(16, 30))

if not os.path.exists('output'):
    os.mkdir('output')

print("Loading model ...")
model = load_model('/Users/faizahkureshi/Desktop/GAIP Project/modelnew.h5')
Q = deque(maxlen=128)
vs = cv2.VideoCapture(0)
f=0
writer = None
(W, H) = (None, None)
count = 0
counter=0
# Register the key press event handler

while True:
  
    # read the next frame from the file
    ret, frame = vs.read()


    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not ret:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 128x128, and then
    # perform mean subtraction


    output = frame.copy()

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (128, 128)).astype("float32")
    frame = frame.reshape(128, 128, 3) / 255

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
#             print("preds",preds)
    Q.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = (preds > 0.50)[0]
    label = i
    print(i)
    text_color = (0, 255, 0) # default : green

    if label: # Violence prob
        f+=1
        text_color = (0, 0, 255) # red

    else:
      f=0

    if label and f>2:
      print("1")
      text = "Violence: {}".format(label)
      FONT = cv2.FONT_HERSHEY_SIMPLEX
      find_loc()

      cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3)

      # check if the video writer is None
      if writer is None:
          # initialize our video writer
          fourcc = cv2.VideoWriter_fourcc(*"MJPG")
          writer = cv2.VideoWriter("output/v_output.avi", fourcc, 30,(W, H), True)

      # write the output frame to disk
      writer.write(output)

      # show the output image
      # show the output image
      plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
      plt.savefig(f"output/output_{counter}.png")
      plt.show()
      counter += 1
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


# Keep the program running until the key is pressed
# release the file pointersq
print("[INFO] cleaning up...")
writer.release()
vs.release()
vs.release()

